In [1]:
import torch
import numpy as np   

In [2]:
def get_randmask(observed_mask, sample_ratio):
        rand_for_mask = torch.rand_like(observed_mask) * observed_mask
        rand_for_mask = rand_for_mask.reshape(len(rand_for_mask), -1)
        for i in range(len(observed_mask)):
            # sample_ratio = 0.2 * np.random.rand()
            sample_ratio = sample_ratio
            num_observed = observed_mask[i].sum().item()
            num_masked = round(num_observed * sample_ratio)
            rand_for_mask[i][rand_for_mask[i].topk(num_masked).indices] = -1
        cond_mask = (rand_for_mask > 0).reshape(observed_mask.shape).float()
        return cond_mask

In [3]:
import sys
import os
sys.path.insert(0, "/home/mafzhang/code/Project/CHLA-Imputation-and-Prediction-for-Bay/")
print(sys.path)
from dataset.dataset_imputation import PRE8dDataset

['/home/mafzhang/code/Project/CHLA-Imputation-and-Prediction-for-Bay/', '/home/mafzhang/code/Project/CHLA-Imputation-and-Prediction-for-Bay/visualization/PRE', '/home/mafzhang/anaconda3/envs/torch211/lib/python311.zip', '/home/mafzhang/anaconda3/envs/torch211/lib/python3.11', '/home/mafzhang/anaconda3/envs/torch211/lib/python3.11/lib-dynload', '', '/home/mafzhang/.local/lib/python3.11/site-packages', '/home/mafzhang/anaconda3/envs/torch211/lib/python3.11/site-packages']


In [4]:
import argparse
parser = argparse.ArgumentParser(description='Imputation')

# args for area and methods
parser.add_argument('--area', type=str, default='PRE', help='which bay area we focus')

# basic args
parser.add_argument('--epochs', type=int, default=500, help='epochs')
parser.add_argument('--batch_size', type=int, default=16, help='batch size')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--wd', type=float, default=1e-4, help='weight decay')
parser.add_argument('--test_freq', type=int, default=500, help='test per n epochs')
parser.add_argument('--embedding_size', type=int, default=32)
parser.add_argument('--hidden_channels', type=int, default=32)
parser.add_argument('--diffusion_embedding_size', type=int, default=64)
parser.add_argument('--side_channels', type=int, default=1)

# args for tasks
parser.add_argument('--in_len', type=int, default=46)
parser.add_argument('--out_len', type=int, default=46)
parser.add_argument('--missing_ratio', type=float, default=0.1)

# args for diffusion
parser.add_argument('--beta_start', type=float, default=0.0001, help='beta start from this')
parser.add_argument('--beta_end', type=float, default=0.2, help='beta end to this')
parser.add_argument('--num_steps', type=float, default=50, help='denoising steps')
parser.add_argument('--num_samples', type=int, default=10, help='n datasets')
parser.add_argument('--schedule', type=str, default='quad', help='noise schedule type')
parser.add_argument('--target_strategy', type=str, default='random', help='mask')

# args for mae
parser.add_argument('--num_heads', type=int, default=8, help='n heads for self attention')
config = parser.parse_args([])

In [6]:
from torch.utils.data import DataLoader
test_dloader = DataLoader(PRE8dDataset(config, mode='test'), 1, shuffle=True)

In [8]:
datas, data_ob_masks, data_gt_masks, labels, label_masks = next(iter(test_dloader))

In [9]:
device = "cuda"
datas, data_ob_masks, data_gt_masks, labels, label_masks = datas.float().to(device), data_ob_masks.to(device), data_gt_masks.to(device), labels.to(device), label_masks.to(device)

In [10]:
model = torch.load("/")

tensor([[[[ 0.0000,  0.0000,  0.0000,  ..., -0.9652, -0.9624, -0.9683]],

         [[ 0.0000,  0.0000,  0.0000,  ..., -1.0080, -1.0046, -1.0187]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         ...,

         [[ 0.0000,  0.0000,  0.0000,  ..., -0.6990, -0.7080, -0.7348]],

         [[ 0.0000,  0.0000,  0.0000,  ..., -0.7318, -0.7289, -0.7141]],

         [[ 0.0000,  0.0000,  0.0000,  ..., -0.8799, -0.8925, -0.8345]]]],
       device='cuda:0')